In [16]:
import requests
from lxml import etree
import csv
import pandas as pd

In [63]:
top250_url = 'https://movie.douban.com/top250?start={}&filter='
movie_name = '名称'
movie_year = '年份'
movie_country = '国家'
movie_type = '类型'
movie_director = '导演'
movie_assess = '评价人数'
movie_score = '评分'
movie_quote = '经典影评'
movie_url = '电影网址'


headers={
    'user-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6478.127 Safari/537.36'
}


In [68]:
movie_num = 0 #电影排名
with open('top250_movie.csv','w',newline='',encoding='utf-8') as f:

    writer = csv.writer(f)

    writer.writerow(['电影排名', '电影名称', '电影年份', '国家', '电影类型', '导演', '经典影评', '评论数', '评分', '电影网址'])

    for pages in range(10):

        movie_content = requests.get(top250_url.format(pages * 25),headers=headers).text
        selector = etree.HTML(movie_content)
        all_list = selector.xpath('//*[@id="content"]/div/div[1]/ol/li')

        for item in all_list:
            movie_name = item.xpath('div/div[2]/div[1]/a/span[1]/text()')[0]
            movie_assess = item.xpath('div/div[2]/div[2]/div/span[4]/text()')[0]
            movie_score = item.xpath('div/div[2]/div[2]/div/span[2]/text()')[0]
            movie_url = item.xpath('div/div[2]/div[1]/a/@href')[0]
            try:
                movie_quote = item.xpath('div/div[2]/div[2]/p[2]/span/text()')[0]
            except IndexError:
                movie_quote = 'None' #有些电影没有影评 做特殊处理
            movie_num += 1
            #处理电影介绍信息
            #电影介绍信息
            movie_intro = item.xpath('div/div[2]/div[2]/p[1]/text()')
            movie_actor_infos = movie_intro[0].lstrip().split('\xa0\xa0\xa0')#将原始字符串按照指定的分隔符切分并去除首尾的空白字符，返回包含分割后部分的列表。
            movie_other_infos = movie_intro[1].lstrip().rstrip().split('\xa0/\xa0')
            #导演信息
            movie_director = movie_actor_infos[0][3:]
            #电影上影年份
            movie_year = movie_other_infos[0]
            #电影的国家
            movie_country = movie_other_infos[1]
            #电影类型
            movie_type = movie_other_infos[2]

            
            writer.writerow([movie_num, movie_name, movie_year, movie_country, movie_type, movie_director, movie_quote, movie_assess, movie_score, movie_url])

            
        

In [45]:
headers={
    'user-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6478.127 Safari/537.36'
}
url = 'https://movie.douban.com/top250?start=0&filter='
data = requests.get(url, headers=headers).text

In [46]:
selector = etree.HTML(data)

In [47]:
#电影标题
film = selector.xpath('//*[@id="content"]/div/div[1]/ol/li')

In [62]:
for div in film:
    title = div.xpath('div/div[2]/div[1]/a/span[1]/text()')[0]
    describe = div.xpath('div/div[2]/div[2]/p[1]/text()')
    url = div.xpath('div/div[2]/div[1]/a/@href')[0]
    comments_nums = div.xpath('div/div[2]/div[2]/div/span[4]/text()')[0]
    comments = div.xpath('div/div[2]/div[2]/p[2]/span/text()')[0]
    

In [61]:
url

['https://movie.douban.com/subject/20495023/']

In [35]:
describe[0]



'\n                            导演: 李·昂克里奇 Lee Unkrich / 阿德里安·莫利纳 Adrian Molina\xa0\xa0\xa0主演: ...'

In [24]:
describe[0].lstrip().split('\xa0\xa0\xa0')

['导演: 弗兰克·德拉邦特 Frank Darabont', '主演: 蒂姆·罗宾斯 Tim Robbins /...']

In [25]:
describe[1]

'\n                            1994\xa0/\xa0美国\xa0/\xa0犯罪 剧情\n                        '

In [26]:
describe[1].lstrip().rstrip().split('\xa0/\xa0')#.lstrip().rstrip()去除首尾空白字符 .split()然后进行分割

['1994', '美国', '犯罪 剧情']

In [27]:
comments_nums

'3033963人评价'

In [28]:
comments_nums[:-3]

'3033963'

In [69]:
df = pd.read_csv("top250_movie.csv",encoding = 'utf-8')

df

,电影排名,电影名称,电影年份,国家,电影类型,导演,经典影评,评论数,评分,电影网址
0,1,肖申克的救赎,1994,美国,犯罪 剧情,弗兰克·德拉邦特 Frank Darabont,希望让人自由。,3033983人评价,9.7,https://movie.douban.com/subject/1292052/
1,2,霸王别姬,1993,中国大陆 中国香港,剧情 爱情 同性,陈凯歌 Kaige Chen,风华绝代。,2242587人评价,9.6,https://movie.douban.com/subject/1291546/
2,3,阿甘正传,1994,美国,剧情 爱情,罗伯特·泽米吉斯 Robert Zemeckis,一部美国近现代史。,2260671人评价,9.5,https://movie.douban.com/subject/1292720/
3,4,泰坦尼克号,1997,美国 墨西哥,剧情 爱情 灾难,詹姆斯·卡梅隆 James Cameron,失去的才是永恒的。,2299851人评价,9.5,https://movie.douban.com/subject/1292722/
4,5,千与千寻,2001,日本,剧情 动画 奇幻,宫崎骏 Hayao Miyazaki,最好的宫崎骏，最好的久石让。,2348828人评价,9.4,https://movie.douban.com/subject/1291561/
...,...,...,...,...,...,...,...,...,...,...
245,246,谍影重重,2002,美国 德国 捷克,动作 悬疑 惊悚,道格·里曼 Doug Liman,哗啦啦啦啦，天在下雨，哗啦啦啦啦，云在哭泣……找自己。,450225人评价,8.6,https://movie.douban.com/subject/1304102/
246,247,阿飞正传,1990,中国香港,犯罪 剧情 爱情,王家卫 Kar Wai Wong,王家卫是一种风格，张国荣是一个代表。,535916人评价,8.5,https://movie.douban.com/subject/1305690/
247,248,朗读者,2008,美国 德国,剧情 爱情,史蒂芬·戴德利 Stephen Daldry,当爱情跨越年龄的界限，它似乎能变得更久远一点，成为一种责任，一种水到渠成的相濡以沫。,467903人评价,8.6,https://movie.douban.com/subject/2213597/
248,249,隐藏人物,2016,美国,剧情 传记 历史,特奥多尔·梅尔菲 Theodore Melfi,NaN,234958人评价,8.9,https://movie.douban.com/subject/26615208/
